#### This Notebook will <li> import the 3 million records from a SQL dataset dataset <li> expand out the column of addresses <li> expand out the international addresses <li> clean up the zip codes <li> extract US addresses inside the international address column <li> identify fake zip codes from the international address column <li> import US postal codes and match & merge state abbreviation with the zip codes <li> store and save the cleaned dataset

In [4]:
#Import packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re

### Import 3 million records using PostgresSQL 

In [24]:
#Import to connect to PostgreSQL 
import psycopg2

#connection
try: 
    conn = psycopg2.connect(host="localhost",database="proc_17_108", user="postgres", password="********")
except:
    print("I am unable to connect to the database")

#SQL Query    
sql = "SELECT * FROM proc_17_108 LIMIT 3000000"
df_3M = pd.read_sql(sql, conn)

# save to CSV file for 3M records
df_3M.to_csv('FCC_comments_3M.csv', index=False)

conn.close()
df_3M.head()

,id,date_submission,contact_email,confirmation_number,text_data,index,filers,addressentity,internationaladdressentity
0,59f8b2bb50b733300f069dc9,2017-07-11T15:53:55.361Z,YaroslavKalinina@pornhub.com,201707112383008210,Allowing broadband providers to throttle their...,filings.2017.7,"[{""name"":""Yaroslav Kalinina""}]","{""city"":"""",""address_line_1"":"""",""state"":"""",""zip...","{""addresstext"":""3575 Eagle Lane\n55792,Virgini..."
1,59f8b2bb50b733300f069dca,2017-07-11T15:53:55.390Z,GalinaMedvedeva@pornhub.com,2017071190138844,We need net neutralityto continue. A free and ...,filings.2017.7,"[{""name"":""Galina Medvedeva""}]","{""city"":"""",""address_line_1"":"""",""state"":"""",""zip...","{""addresstext"":""2245 Lodgeville Road\n55415,Mi..."
2,59f8b2bb50b733300f069dcb,2017-07-11T15:53:55.449Z,PhilemonButusov@pornhub.com,20170711233022335,make sure net neutrality does not dissapear. I...,filings.2017.7,"[{""name"":""Philemon Butusov""}]","{""city"":"""",""address_line_1"":"""",""state"":"""",""zip...","{""addresstext"":""4702 Ashmor Drive\n55802,Dulut..."
3,59f8b2bb50b733300f069dcc,2017-07-11T15:53:55.473Z,BarbaraUlyanova@pornhub.com,201707112678519835,We need net neutralityto continue. A free and ...,filings.2017.7,"[{""name"":""Barbara Ulyanova""}]","{""city"":"""",""address_line_1"":"""",""state"":"""",""zip...","{""addresstext"":""2323 Lodgeville Road\n55406,Mi..."
4,59f8b2bb50b733300f069dcd,2017-07-11T15:53:55.512Z,MatildaZhdanova@pornhub.com,201707111845703590,Please save the internet from the corporations...,filings.2017.7,"[{""name"":""Matilda Zhdanova""}]","{""city"":"""",""address_line_1"":"""",""state"":"""",""zip...","{""addresstext"":""2135 Lena Lane\n39401,Hattiesb..."


### Explore the dataset <br>
Notice the pornhub email addresses above, the identical submission times down to the second, and the Russian names? There is also something odd about contents of the International Address Entity column. We will investigate all these.

In [8]:
df_3M = pd.read_csv('FCC_comments_3M.csv', encoding='utf8')

A section of my EDA has to do with zipcode analysis. To evaluate the zipcodes from the domestic address columns I will remove any NaN.  

In [3]:
# Since most of the work will happen on addressentity, remove records with NaN for addressentity
# Explore addressentity NaN records

delete_address = df_3M[df_3M['addressentity'].isnull()]
delete_address[['id','text_data','addressentity']].head()

,id,text_data,addressentity
1101755,59f8c83250b733300f176af3,NaN,NaN
1917656,59f8d93b50b733300f23f38c,IF YOU WANT CHINA TO TAKE OVER YOUR INTERNET B...,NaN


In [12]:
# Remove the 2 records with NaN for addressentity
df_3M = df_3M[~df_3M['addressentity'].isnull()]
df_3M.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2999998 entries, 0 to 2999999
Data columns (total 9 columns):
id                            object
date_submission               object
contact_email                 object
confirmation_number           float64
text_data                     object
index                         object
filers                        object
addressentity                 object
internationaladdressentity    object
dtypes: float64(1), object(8)
memory usage: 228.9+ MB


Expand dictionaries in addressentity field into their own dataframe 

In [5]:
#expand dictionary using eval() in the addressentity column to get the keys/values
#address_line_1
#address_line_1
#city
#state
#zip4
#zipcode

df_3M["addressentity"] = df_3M["addressentity"].apply(lambda x : dict(eval(x)))

#then apply pd.Series to turn the values into a dataframe
df_address = df_3M["addressentity"].apply(pd.Series)

In [17]:
#explore the expanded address dataframe
df_address.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2999998 entries, 0 to 2999999
Data columns (total 11 columns):
address_line_1                object
address_line_2                object
city                          object
contact_email                 object
express_comment               float64
internationaladdressentity    object
state                         object
text_data                     object
zip4                          object
zip_code                      object
zipcode                       object
dtypes: float64(1), object(10)
memory usage: 274.7+ MB


There are more fields here than there are supposed to be for a United States address. The inclusion of following keys from the dictionary are surprises: <li> an <i> additional </i> contact_email <li> an express_comment column <li> an <i>additional</i>  text_data column <li> <i>2</i> zipcode columns <li> an <i>additional</i>  internationaladdressentity column

Let's investigate the peculiar 'express_comment' column, which is of type float

In [10]:
#select rows where express comment is not null
express_comment = df_address[~df_address['express_comment'].isnull()]

#display rows without street address details
express_comment[['city','contact_email','express_comment','zip_code','internationaladdressentity','zipcode']].head()

'johndoe@gmail.com'? It appears we have some falsified entries.

In [131]:
print('Number of rows where contat_email is "johndoe@gmail.com" = {}.'.format(\
len(df_address[df_address['contact_email']=='johndoe@gmail.com'])))

Number of rows where contat_email is "johndoe@gmail.com" = 15027.


And the internationaladdressentity column?

In [130]:
#select rows where internationaladdressentity is not null
print('Number of rows where internationaladdressentity is not blank = {}.'.format(\
len(df_address[~df_address['internationaladdressentity'].isnull()]\
    [['city','contact_email','express_comment','zip_code','internationaladdressentity','zipcode']])))

Number of rows where internationaladdressentity is not blank = 15027.


In [132]:
#select rows where internationaladdressentity is not null & johndoe
print('Number of rows where email is johndoe AND internationaladdressentity is not blank = {}.'.format(\
len(df_address[(~df_address['internationaladdressentity'].isnull()) & \
               (df_address['contact_email']=='johndoe@gmail.com')]\
    [['city','contact_email','express_comment','zip_code','internationaladdressentity','zipcode']])))

Number of rows where email is johndoe AND internationaladdressentity is not blank = 15027.


So all the 15,027 rows are good candidates for being fake filings where the field: addressentity was stuffed with additional keys, a fake email and similar submission times.

Also the pair of zipcodes might cause some confusion. Let's see if there any null 'zip_code'?

In [57]:
#select rows where zipcode is not null
df_address[~df_address['zipcode'].isnull()][['city','contact_email','express_comment','text_data','zip_code','zipcode']].head(10)

,city,contact_email,express_comment,text_data,zip_code,zipcode
7228,ENCINITAS,NaN,NaN,NaN,NaN,92024
7851,ENCINITAS,NaN,NaN,NaN,NaN,92024
8368,ENCINITAS,NaN,NaN,NaN,NaN,92024
16373,WORCESTER,NaN,NaN,NaN,NaN,01609
19144,ENCINITAS,NaN,NaN,NaN,NaN,92024
35902,ENCINITAS,NaN,NaN,NaN,NaN,92024
35907,ENCINITAS,NaN,NaN,NaN,NaN,92024
38055,ENCINITAS,NaN,NaN,NaN,NaN,92024
38060,ENCINITAS,NaN,NaN,NaN,NaN,92024
38087,ENCINITAS,NaN,NaN,NaN,NaN,92024


The duplicate entries of this zipcode also indicate a fake filing (the redacted addresses from this all uppercase ENCITAS address are identical). Lets see what some of these records look like in the original data set:  

In [75]:
df_3M.loc[[35902,38055,38060,38087]][['date_submission','text_data']]

,date_submission,text_data
35902,2017-07-11T21:33:07.293Z,To whom it may concern:\r\nI would like my vid...
38055,2017-07-11T21:32:15.635Z,To whom it may concern:\r\nI would like my vid...
38060,2017-07-11T21:32:31.579Z,To whom it may concern:\r\nI would like my vid...
38087,2017-07-11T21:32:33.863Z,To whom it may concern:\r\nI would like my vid...


Notice that the same text_data and similiarity in time of filing, these are within 1 minute. Also, not shown, the filing address(redacted) and filing email(redacted) are identical

How many of these in the df_address have text_data entries?

In [94]:
print("Non null text_data rows: {}, the same as 'johndoe@gmail' and 'express_comment'."\
        .format(len(df_address[~df_address['text_data'].isnull()])))
df_address[~df_address['text_data'].isnull()]\
[['city','contact_email','express_comment','text_data','zip_code','zipcode']].head(20)

Non null text_data rows: 15027, the same as 'johndoe@gmail' and 'express_comment'.


,city,contact_email,express_comment,text_data,zip_code,zipcode
2287451,Plain City,johndoe@gmail.com,1.0,I support Net Neutrality,43064,NaN
2287545,Boise,johndoe@gmail.com,1.0,Please do not over rule net neutrality as that...,83716,NaN
2287588,Kenneth City,johndoe@gmail.com,1.0,,33709,NaN
2287598,Kaysville,johndoe@gmail.com,1.0,,84037,NaN
2287638,Deltona,johndoe@gmail.com,1.0,,32725,NaN
2287693,Pacific Junction,johndoe@gmail.com,1.0,,51561,NaN
2287824,truckee,johndoe@gmail.com,1.0,THere will be a revolution by the people if th...,96161,NaN
2288016,Sandy Springs,johndoe@gmail.com,1.0,This is an important issue for me and my family.,30350,NaN
2288055,Tucson,johndoe@gmail.com,1.0,Please keep Net Neutrality don't listen to the...,85739,NaN
2288967,Dundee,johndoe@gmail.com,1.0,I believe strongly in Net Neutrality. I belon...,97115,NaN


In [19]:
df_address.tail()

,address_line_1,address_line_2,city,express_comment,state,zip_code
2999993,794 Sheridan Ave.,NaN,Vineland,NaN,NJ,08361
2999994,xxxxxxx,NaN,Anaheim,NaN,CA,92801
2999995,53 sinclair dr,NaN,greenlawn,NaN,NY,11740
2999996,119 Arcaro drive,NaN,Greensboro,NaN,NC,27455
2999997,2621 Fulton Ave; Apt#61,NaN,Sacramento,NaN,CA,95821


Having two zip code fields is confusing. To what extent is there an entry in <i>both</i> columns?

In [124]:
print("Non null zipcode & zip_code rows = {}. There is no overlap between the 2 zip code columns.".format(len\
       (df_address[~df_address['zipcode'].isnull() & ~df_address['zip_code'].isnull()])))

#show rows where both zip_code and zipcode are present
df_address[~df_address['zipcode'].isnull() & ~df_address['zip_code'].isnull()]\
[['city','contact_email','express_comment','text_data','zip_code','zipcode']].head()

Non null zipcode & zip_code rows = 0. There is no overlap between the 2 zip code columns.


,city,contact_email,express_comment,text_data,zip_code,zipcode


### Data Wrangling <br>


We can now clean up the df_address dataframe, and then merge it back to the main dataframe to continue the analysis.

Lets start by merging the zip code fields into a new column and adding leading 0's and deleting the old columns

In [ ]:
#merge zipcodes into new column and fill NA cells
df_address['zipcode_blended'] = df_address['zip_code'].fillna('') + df_address['zipcode'].fillna('')

In [140]:
#drop unnecessary columns
df_address.drop(['zip_code','zipcode'],axis=1,inplace=True)

In [141]:
# add leading zeros to make all zip codes 5 digits as a string
df_address['zipcode_blended'] = df_address['zipcode_blended'].str.zfill(5)

Lets drop the zip4 field and internationaladdressentity columns. In the first case we do not need the detail that the zip4 provides. For the internationaladdressentity field that, like the express_comment column, is only present when the contact_email is 'johndoe'

In [133]:
#drop unnecessary columns
df_address.drop(['internationaladdressentity','zip4'],axis=1,inplace=True)

Lets rename contact_email and text_data so there are no issues when this is merged back to the main dataset, and zip code.

In [143]:
# rename columns
df_address=df_address.rename(columns = {'contact_email':'contact_emailx','text_data':'text_datax','zipcode_blended':'zip_code'})

In [146]:
df_address.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2999998 entries, 0 to 2999999
Data columns (total 8 columns):
address_line_1     object
address_line_2     object
city               object
contact_emailx     object
express_comment    float64
state              object
text_datax         object
zip_code           object
dtypes: float64(1), object(7)
memory usage: 206.0+ MB


In [306]:
df_address.iloc[:,1:].tail()

,address_line_2,city,contact_emailx,express_comment,state,text_datax,zip_code
2999995,,Vineland,NaN,NaN,NJ,NaN,08361
2999996,,Anaheim,NaN,NaN,CA,NaN,92801
2999997,,greenlawn,NaN,NaN,NY,NaN,11740
2999998,,Greensboro,NaN,NaN,NC,NaN,27455
2999999,,Sacramento,NaN,NaN,CA,NaN,95821


From the main dataset, lets explore the internationaladdressentity column <br> These address appear legitimate but they do not exist.

In [150]:
df_expanded_nonUS = df_3M['internationaladdressentity'].apply(pd.Series)
df_expanded_nonUS.head()

,0
0,"{""addresstext"":""3575 Eagle Lane\n55792,Virgini..."
1,"{""addresstext"":""2245 Lodgeville Road\n55415,Mi..."
2,"{""addresstext"":""4702 Ashmor Drive\n55802,Dulut..."
3,"{""addresstext"":""2323 Lodgeville Road\n55406,Mi..."
4,"{""addresstext"":""2135 Lena Lane\n39401,Hattiesb..."


Lets rename the column, then expand out the the 5 digit zipcodes using regex and take a look at some in more detail. I suspect the zipcodes are for non existant addresses.

In [166]:
#rename column
df_expanded_nonUS.columns = ['intl_address']

In [199]:
# define function to extract zip code from free form field "international address entity"
def get_zip (y):
    """Takes 5 digits from string field for zip code - 
    returns false value if regex expression does not find 5 digit sequence"""
    reg = re.compile('\d{5}')
    z = reg.findall(y)
    if len(z) == 0:
        #return "no zip code"
        return float('nan')
    else:
        return z[0] 

In [200]:
# apply zip code funtion to extract 5 digit zip codes or replace with NaN
df_expanded_nonUS['probably_fake_zip'] = df_expanded_nonUS['intl_address'].\
                                        apply(lambda x: get_zip(x) if type(x) == str else x)

In [236]:
#inspect first cell
df_expanded_nonUS.loc[2]['intl_address']

'{"addresstext":"4702 Ashmor Drive\\n55802,Duluth, United States"}'

Notice the following details of this address <li> the escape character ('\\n') <li> the missing State abreviation <li> United States spelled out in full <br> These are not the way most Americans would enter address information, espcially if they went to the trouble to provide street address and zipcode, they would not omit the state. <br><br> This address, by the way, does not exist.

Another address is in Cyrillic, nothing odd about that in the international address field. Except that it is repeated mulitple times.

In [237]:
df_expanded_nonUS.loc[340000:340003]['intl_address']

340000    {"addresstext":"улица Полевая \n кв. 391 \n Че...
340001    {"addresstext":"улица Полевая \n кв. 391 \n Че...
340002    {"addresstext":"улица Полевая \n кв. 391 \n Че...
340003    {"addresstext":"улица Полевая \n кв. 391 \n Че...
Name: intl_address, dtype: object

No 5 digit sequence exists here, so NaN is returned as expected.

In [239]:
df_expanded_nonUS[340000:340004]

,intl_address,probably_fake_zip
340000,"{""addresstext"":""улица Полевая \n кв. 391 \n Че...",NaN
340001,"{""addresstext"":""улица Полевая \n кв. 391 \n Че...",NaN
340002,"{""addresstext"":""улица Полевая \n кв. 391 \n Че...",NaN
340003,"{""addresstext"":""улица Полевая \n кв. 391 \n Че...",NaN


What do these row numbers return for the df_address and the main dataset df_3M? <li> A zip code of 00000 <li> contact emails from pornhub and hurra.de accounts <li> identically timed submissions

In [219]:
df_address[340000:340004]

,address_line_1,address_line_2,city,contact_emailx,express_comment,state,text_datax,zip_code
340000,,NaN,,NaN,NaN,,NaN,00000
340001,,NaN,,NaN,NaN,,NaN,00000
340002,,NaN,,NaN,NaN,,NaN,00000
340003,,NaN,,NaN,NaN,,NaN,00000


In [296]:
df_3M[340000:340004]

,id,date_submission,contact_email,confirmation_number,text_data,index,filers,addressentity,internationaladdressentity
340000,59f8b9cb50b733300f0bd155,2017-07-12T03:04:06.043Z,TrifonAbramovich@pornhub.com,2.017071e+17,The Open Internet rules (net neutrality) are e...,filings.2017.7,"[{""name"":""Trifon Abramovich""}]","{'city': '', 'address_line_1': '', 'state': ''...","{""addresstext"":""улица Полевая \n кв. 391 \n Че..."
340001,59f8b9cb50b733300f0bd156,2017-07-12T03:04:06.088Z,TeresaSukhorukova@hurra.de,2.017071e+16,On July 12 is the Protect Net Neutrality Day o...,filings.2017.7,"[{""name"":""Teresa Sukhorukova""}]","{'city': '', 'address_line_1': '', 'state': ''...","{""addresstext"":""улица Полевая \n кв. 391 \n Че..."
340002,59f8b9cb50b733300f0bd157,2017-07-12T03:04:06.166Z,RinatTretyakov@hurra.de,2.017071e+16,On July 12 is the Protect Net Neutrality Day o...,filings.2017.7,"[{""name"":""Rinat Tretyakov""}]","{'city': '', 'address_line_1': '', 'state': ''...","{""addresstext"":""улица Полевая \n кв. 391 \n Че..."
340003,59f8b9cb50b733300f0bd158,2017-07-12T03:04:06.331Z,MuslimShcherbakov@pornhub.com,2.017071e+16,The Open Internet rules (net neutrality) are e...,filings.2017.7,"[{""name"":""Muslim Shcherbakov""}]","{'city': '', 'address_line_1': '', 'state': ''...","{""addresstext"":""улица Полевая \n кв. 391 \n Че..."


How many of these records have that address written in Cyrillic?

In [240]:
#Isolate the address in Cyrillic
x = df_expanded_nonUS.loc[340000]['intl_address']

In [245]:
#Print the number of rows
print('Number of rows that have the same address written in Cyrillic: {}!'\
      .format(len(df_expanded_nonUS[df_expanded_nonUS['intl_address']==x])))

Number of rows that have the same address written in Cyrillic: 444715!


What are the most common zip codes present in our dataset? The 00000s are from the filling out the zip code string.

In [252]:
print('Zip Code    QTY')
print(df_address['zip_code'].value_counts(sort=True, dropna=False).head(15))

Zip Code    QTY
00000    694675
94110      1376
97115      1353
10025      1315
95060      1230
98103      1226
10011      1214
14850      1197
11229      1184
11201      1172
48103      1134
97402      1133
11215      1087
97202      1081
60647      1065
Name: zip_code, dtype: int64


Merge the df_address data to the df_3M

In [308]:
df = pd.concat([df_3M, df_address], axis=1)

In [309]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2999998 entries, 0 to 2999999
Data columns (total 17 columns):
id                            object
date_submission               object
contact_email                 object
confirmation_number           float64
text_data                     object
index                         object
filers                        object
addressentity                 object
internationaladdressentity    object
address_line_1                object
address_line_2                object
city                          object
contact_emailx                object
express_comment               float64
state                         object
text_datax                    object
zip_code                      object
dtypes: float64(2), object(15)
memory usage: 492.0+ MB


In [310]:
#merge text_data column and fill NA cells
df['text_data'] = df['text_data'].fillna('') + df['text_datax'].fillna('')

In [311]:
#merge text_data column and fill NA cells
df['contact_email'] = df['contact_email'].fillna('') + df['contact_emailx'].fillna('')

In [337]:
#drop unnecessary columns
df.drop(['text_datax','contact_emailx'],axis=1,inplace=True)

In [314]:
#save the progress
df_3M.to_csv('df_3M.csv',index=False)
df_address.to_csv('df_address.csv',index=False)
df.to_csv('df.csv',index=False)
df_expanded_nonUS.to_csv('df_expanded_nonUS.csv',index=False)

Lets clean up the expanded international address dataframe so only US address zip codes are present and drop zip from the remaining overseas addresses.  

In [367]:
#US addresses in international address field
#with 5 digits in address that are likely fake
df_expanded_nonUS['fake_zip'] = df_expanded_nonUS[ \
                  (df_expanded_nonUS['intl_address']!='{"addresstext":""}') & \ #empty contents
                  (~df_expanded_nonUS['intl_address'].isnull()) &\ #null contents
                  (df_expanded_nonUS['intl_address']!='{}') & \ #'blank contents
                  (df_expanded_nonUS['intl_address'].str.contains("United States")) & \ #United States specified- though no cities
                  (~df_expanded_nonUS['probably_fake_zip'].isnull())] \ #has an extracted 5 digit string zip code
                 ['probably_fake_zip'] # copy contents to fake_zip column

In [370]:
df_expanded_nonUS.head()

,intl_address,probably_fake_zip,fake_zip
0,"{""addresstext"":""3575 Eagle Lane\n55792,Virgini...",55792,55792
1,"{""addresstext"":""2245 Lodgeville Road\n55415,Mi...",55415,55415
2,"{""addresstext"":""4702 Ashmor Drive\n55802,Dulut...",55802,55802
3,"{""addresstext"":""2323 Lodgeville Road\n55406,Mi...",55406,55406
4,"{""addresstext"":""2135 Lena Lane\n39401,Hattiesb...",39401,39401


In [371]:
#drop unnecessary columns
df_expanded_nonUS.drop(['probably_fake_zip'],axis=1,inplace=True)

In [375]:
df_expanded_nonUS[340000:340010]

,intl_address,fake_zip
340000,"{""addresstext"":""улица Полевая \n кв. 391 \n Че...",NaN
340001,"{""addresstext"":""улица Полевая \n кв. 391 \n Че...",NaN
340002,"{""addresstext"":""улица Полевая \n кв. 391 \n Че...",NaN
340003,"{""addresstext"":""улица Полевая \n кв. 391 \n Че...",NaN
340004,"{""addresstext"":""улица Полевая \n кв. 391 \n Че...",NaN
340005,"{""addresstext"":""улица Полевая \n кв. 391 \n Че...",NaN
340006,"{""addresstext"":""улица Полевая \n кв. 391 \n Че...",NaN
340007,"{""addresstext"":""улица Полевая \n кв. 391 \n Че...",NaN
340008,"{""addresstext"":""улица Полевая \n кв. 391 \n Че...",NaN
340009,NaN,NaN


Merge the international address fake zip information with the main dataset df

In [377]:
df = pd.concat([df, df_expanded_nonUS], axis=1)

Lets remove some more unnecessary fields

In [381]:
#drop unnecessary columns
df.drop(['confirmation_number','index','address_line_2','internationaladdressentity'],axis=1,inplace=True)

In [44]:
df.head()

,id,date_submission,contact_email,text_data,filers,addressentity,address_line_1,city,express_comment,state,zip_code,intl_address,fake_zip
0,59f8b2bb50b733300f069dc9,2017-07-11T15:53:55.361Z,YaroslavKalinina@pornhub.com,Allowing broadband providers to throttle their...,"[{""name"":""Yaroslav Kalinina""}]","{'city': '', 'address_line_1': '', 'state': ''...",NaN,NaN,NaN,NaN,NaN,"{""addresstext"":""3575 Eagle Lane\n55792,Virgini...",55792
1,59f8b2bb50b733300f069dca,2017-07-11T15:53:55.390Z,GalinaMedvedeva@pornhub.com,We need net neutralityto continue. A free and ...,"[{""name"":""Galina Medvedeva""}]","{'city': '', 'address_line_1': '', 'state': ''...",NaN,NaN,NaN,NaN,NaN,"{""addresstext"":""2245 Lodgeville Road\n55415,Mi...",55415
2,59f8b2bb50b733300f069dcb,2017-07-11T15:53:55.449Z,PhilemonButusov@pornhub.com,make sure net neutrality does not dissapear. I...,"[{""name"":""Philemon Butusov""}]","{'city': '', 'address_line_1': '', 'state': ''...",NaN,NaN,NaN,NaN,NaN,"{""addresstext"":""4702 Ashmor Drive\n55802,Dulut...",55802
3,59f8b2bb50b733300f069dcc,2017-07-11T15:53:55.473Z,BarbaraUlyanova@pornhub.com,We need net neutralityto continue. A free and ...,"[{""name"":""Barbara Ulyanova""}]","{'city': '', 'address_line_1': '', 'state': ''...",NaN,NaN,NaN,NaN,NaN,"{""addresstext"":""2323 Lodgeville Road\n55406,Mi...",55406
4,59f8b2bb50b733300f069dcd,2017-07-11T15:53:55.512Z,MatildaZhdanova@pornhub.com,Please save the internet from the corporations...,"[{""name"":""Matilda Zhdanova""}]","{'city': '', 'address_line_1': '', 'state': ''...",NaN,NaN,NaN,NaN,NaN,"{""addresstext"":""2135 Lena Lane\n39401,Hattiesb...",39401


Convert fake_zip to integer from float, convert to string and zero fill for 5 characters.

In [61]:
#amend fake_zip field where it is not null
df['fake_zip']=df[~df['fake_zip'].isnull()]['fake_zip'].astype(int).astype(str).str.zfill(5)

In [11]:
print('Fake Zip QTY')
print(df['fake_zip'].value_counts(sort=True).head(15))

Fake Zip QTY
39201    473
90017    291
39211    226
49503    187
48075    152
39213    127
19108    124
30303    102
94612    101
21202     98
10016     97
10013     96
07102     94
39402     93
10011     93
Name: fake_zip, dtype: int64


Confirm that there is no overlap in zip_code and fake_zip rows

In [50]:
#Number of rows where there is both a zip_code and fake_zip entry
len(df[~df['zip_code'].isnull() & ~df['fake_zip'].isnull()])

0

Merge the zip code and fake zip fields. 

In [13]:
#merge zip codes column and fill NA cells
df['combined_zip'] =df['zip_code'].fillna(df['fake_zip'])

In [7]:
df['combined_zip']=df[~df['combined_zip'].isnull()]['combined_zip'].astype(int).astype(str).str.zfill(5)

Create a flagged field for the fake_zip for use in identify fake fillings

In [18]:
df['flagged_zip'] = df['fake_zip'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [ ]:
#drop more unnecessary columns
df.drop(['fake_zip','zip_code','addressentity'],axis=1,inplace=True)

In [2]:
df.head()

,id,date_submission,contact_email,text_data,filers,address_line_1,city,express_comment,state,intl_address,combined_zip,flagged_zip
0,59f8b2bb50b733300f069dc9,2017-07-11T15:53:55.361Z,YaroslavKalinina@pornhub.com,Allowing broadband providers to throttle their...,"[{""name"":""Yaroslav Kalinina""}]",NaN,NaN,NaN,NaN,"{""addresstext"":""3575 Eagle Lane\n55792,Virgini...",55792,1
1,59f8b2bb50b733300f069dca,2017-07-11T15:53:55.390Z,GalinaMedvedeva@pornhub.com,We need net neutralityto continue. A free and ...,"[{""name"":""Galina Medvedeva""}]",NaN,NaN,NaN,NaN,"{""addresstext"":""2245 Lodgeville Road\n55415,Mi...",55415,1
2,59f8b2bb50b733300f069dcb,2017-07-11T15:53:55.449Z,PhilemonButusov@pornhub.com,make sure net neutrality does not dissapear. I...,"[{""name"":""Philemon Butusov""}]",NaN,NaN,NaN,NaN,"{""addresstext"":""4702 Ashmor Drive\n55802,Dulut...",55802,1
3,59f8b2bb50b733300f069dcc,2017-07-11T15:53:55.473Z,BarbaraUlyanova@pornhub.com,We need net neutralityto continue. A free and ...,"[{""name"":""Barbara Ulyanova""}]",NaN,NaN,NaN,NaN,"{""addresstext"":""2323 Lodgeville Road\n55406,Mi...",55406,1
4,59f8b2bb50b733300f069dcd,2017-07-11T15:53:55.512Z,MatildaZhdanova@pornhub.com,Please save the internet from the corporations...,"[{""name"":""Matilda Zhdanova""}]",NaN,NaN,NaN,NaN,"{""addresstext"":""2135 Lena Lane\n39401,Hattiesb...",39401,1


Reading in the US Postal Codes dataset

In [7]:
# import zip codes, cities, states and geo coordinates 
us_postal_codes = pd.read_csv('us_postal_codes_as_str.csv', encoding='utf8')
us_postal_codes.head()

,Zip_Code_num,Place_Name,State,State_Abbreviation,County,Latitude,Longitude,Zip_Code_str
0,501,Holtsville,New York,NY,Suffolk,40.8154,-73.0451,501
1,544,Holtsville,New York,NY,Suffolk,40.8154,-73.0451,544
2,1001,Agawam,Massachusetts,MA,Hampden,42.0702,-72.6227,1001
3,1002,Amherst,Massachusetts,MA,Hampshire,42.3671,-72.4646,1002
4,1003,Amherst,Massachusetts,MA,Hampshire,42.3919,-72.5248,1003


In [8]:
#convert zip code integer to string and fill out to 5 places
us_postal_codes['Zip_Code_str'] = us_postal_codes['Zip_Code_str'].astype(str).str.zfill(5)
us_postal_codes.head()

,Zip_Code_num,Place_Name,State,State_Abbreviation,County,Latitude,Longitude,Zip_Code_str
0,501,Holtsville,New York,NY,Suffolk,40.8154,-73.0451,00501
1,544,Holtsville,New York,NY,Suffolk,40.8154,-73.0451,00544
2,1001,Agawam,Massachusetts,MA,Hampden,42.0702,-72.6227,01001
3,1002,Amherst,Massachusetts,MA,Hampshire,42.3671,-72.4646,01002
4,1003,Amherst,Massachusetts,MA,Hampshire,42.3919,-72.5248,01003


In [9]:
# merge dfs of US zipcodes with us_postal_codes to create an appropriate mapping for state_abbreviations
df = pd.merge(df, us_postal_codes, left_on='combined_zip',right_on='Zip_Code_str', how='left')

In [13]:
#drop more unnecessary columns
df.drop(['Latitude','Longitude','County','State','Zip_Code_str','Zip_Code_num'],axis=1,inplace=True)

In [15]:
df.to_csv('df_5.csv',index=False)

In [16]:
%store df

Stored 'df' (DataFrame)


In [14]:
df.tail()

,id,date_submission,contact_email,text_data,filers,address_line_1,city,express_comment,state,intl_address,combined_zip,flagged_zip,Zip_Code_num,Place_Name,State_Abbreviation
2999993,59f8eeec50b733300f347152,2017-07-17T23:16:41+00:00,NaN,Cable and phone companies provide access to th...,"[{""name"":""Ismael Rodriguez""}]",794 Sheridan Ave.,Vineland,NaN,NJ,"{""addresstext"":""""}",08361,0,8361.0,Vineland,NJ
2999994,59f8eeec50b733300f3482a5,2017-07-17T23:19:39+00:00,NaN,Net Neutrality is not negotiable. It's essenti...,"[{""name"":""Angelica Munoz""}]",xxxxxxx,Anaheim,NaN,CA,"{""addresstext"":""""}",92801,0,92801.0,Anaheim,CA
2999995,59f8eeec50b733300f347153,2017-07-17T23:16:41+00:00,NaN,Cable and phone companies provide access to th...,"[{""name"":""Richard Krueger""}]",53 sinclair dr,greenlawn,NaN,NY,"{""addresstext"":""""}",11740,0,11740.0,Greenlawn,NY
2999996,59f8eeec50b733300f347154,2017-07-17T23:16:41+00:00,NaN,Cable and phone companies provide access to th...,"[{""name"":""Kathi Brinkman""}]",119 Arcaro drive,Greensboro,NaN,NC,"{""addresstext"":""""}",27455,0,27455.0,Greensboro,NC
2999997,59f8eeec50b733300f347155,2017-07-17T23:16:41+00:00,NaN,Cable and phone companies provide access to th...,"[{""name"":""Sandara Stotenburg""}]",2621 Fulton Ave; Apt#61,Sacramento,NaN,CA,"{""addresstext"":""""}",95821,0,95821.0,Sacramento,CA
